In [130]:
# import library
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import streamlit as st
from pandas.tseries.offsets import DateOffset

In [140]:
def load_data():

    # load data
    df_vid = pd.read_csv('resource/Video_Performance_Over_Time.csv')

    # dropping row Total to avoid issue with calculations
    df_agg = pd.read_csv('resource/Aggregated_Metrics_By_Video.csv').iloc[1:,:]

    df_agg_sub = pd.read_csv('resource/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')

    df_com = pd.read_csv('resource/All_Comments_Final.csv')

    # converting all columns to uppercase
    for df in [df_agg,df_vid,df_agg_sub, df_com]:
        df.columns = df.columns.str.upper()

    # remove \xad
    df_agg.columns = df_agg.columns.str.replace('\xad','')

    # drop Nan value
    df_agg.dropna(inplace=True)

    # convert date to datetype

    df_agg['VIDEO PUBLISH TIME'] = pd.to_datetime(df_agg['VIDEO PUBLISH TIME'], format='mixed')

    # rename column

    df_agg.rename(columns={'VIDEO PUBLISH TIME': 'PUBLISH DATE', 'COMMENTS ADDED' : 'COMMENTS'}, inplace=True)
        
    df_agg['AVERAGE VIEW DURATION'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))

    # create new column for df_agg['AVERAGE VIEW SECONDS'
    df_agg['AVERAGE VIEW SECONDS'] = df_agg['AVERAGE VIEW DURATION'].apply(lambda x: x.second + x.minute * 60 + x.hour * 60 * 60)

    # engagement ration, every engagement a view could do divided by the number of viewrs
    df_agg['ENGAGEMENT RATIO'] = (df_agg['SHARES'] + df_agg['LIKES'] + df_agg['DISLIKES'] + df_agg['COMMENTS']) / df_agg['VIEWS'] 

    # ratio of views to subscribers gained
    df_agg['VIEW TO SUBSCRIBER RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS GAINED'] # how views does it take to gain a subscriber

    # ratio of views to subscribers lost, 
    df_agg['VIEW TO SUBSCRIBER LOST RATIO'] = df_agg['VIEWS'] / df_agg['SUBSCRIBERS LOST'] # how views does it take to lose a subscriber

    # sort data by 'VIDEO PUBLISH TIME'
    df_agg.sort_values(by = 'PUBLISH DATE', ascending=False, inplace=True)

    # CONVERTING DATE to datetime
    df_vid['DATE'] = pd.to_datetime(df_vid['DATE'], format='mixed')

    df_com['DATE'] = pd.to_datetime(df_com['DATE'])
    # renaming vidId
    df_com.rename(columns={'VIDID':'VIDEO'}, inplace=True)
    # removing '_'
    df_time.columns = df_time.columns.str.replace('_', ' ')

    # create dataframe
    return df_vid, df_agg, df_agg_sub, df_com

df_vid, df_agg, df_agg_sub, df_time = load_data()

In [144]:
for i in [df_vid, df_agg, df_agg_sub, df_time]:
    print(i.columns.tolist())
    print('*'*40)

['DATE', 'VIDEO TITLE', 'EXTERNAL VIDEO ID', 'VIDEO LENGTH', 'THUMBNAIL LINK', 'VIEWS', 'VIDEO LIKES ADDED', 'VIDEO DISLIKES ADDED', 'VIDEO LIKES REMOVED', 'USER SUBSCRIPTIONS ADDED', 'USER SUBSCRIPTIONS REMOVED', 'AVERAGE VIEW PERCENTAGE', 'AVERAGE WATCH TIME', 'USER COMMENTS ADDED']
****************************************
['VIDEO', 'VIDEO TITLE', 'PUBLISH DATE', 'COMMENTS', 'SHARES', 'DISLIKES', 'LIKES', 'SUBSCRIBERS LOST', 'SUBSCRIBERS GAINED', 'RPM (USD)', 'CPM (USD)', 'AVERAGE PERCENTAGE VIEWED (%)', 'AVERAGE VIEW DURATION', 'VIEWS', 'WATCH TIME (HOURS)', 'SUBSCRIBERS', 'YOUR ESTIMATED REVENUE (USD)', 'IMPRESSIONS', 'IMPRESSIONS CLICK-THROUGH RATE (%)', 'AVERAGE VIEW SECONDS', 'ENGAGEMENT RATIO', 'VIEW TO SUBSCRIBER RATIO', 'VIEW TO SUBSCRIBER LOST RATIO']
****************************************
['VIDEO TITLE', 'EXTERNAL VIDEO ID', 'VIDEO LENGTH', 'THUMBNAIL LINK', 'COUNTRY CODE', 'IS SUBSCRIBED', 'VIEWS', 'VIDEO LIKES ADDED', 'VIDEO DISLIKES ADDED', 'VIDEO LIKES REMOVED', 'USER

In [154]:
df_merge1 = pd.merge(left=df_vid.loc[:['VIDEO TITLE', 'EXTERNAL VIDEO ID', 'VIEWS', 'VIDEO LENGTH']], right=df_agg_sub, how='inner', on='EXTERNAL VIDEO ID')
df_merge1.head(1)

InvalidIndexError: ['VIDEO TITLE', 'EXTERNAL VIDEO ID', 'VIEWS', 'VIDEO LENGTH']

In [153]:
pd.merge(left=df_merge1[1:], right=df_time, how='inner')

ValueError: You are trying to merge on datetime64[ns] and datetime64[ns, UTC] columns for key 'DATE'. If you wish to proceed you should use pd.concat